# Example
of the last added functionalities in onlineSG

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import source.players as pl

In [3]:
import pkgutil

In [5]:
import source.parsers as parsers

In [1]:
import sys
sys.path.append('../')  # uncomment to add the path to the onlineSG folder 
# You have to change the path if you move this file.
# Alternatevely add the onlineSG folder to your PYTHONPATH
# e.g. On Linux add this line on .bashrc:
# export PYTHONPATH="<absolute-path-to-onlineSG>:PYTHONPATH"
# then you can use onlineSG library from everywhere

In [5]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.util as util

import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
from copy import deepcopy


In [2]:
lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [4]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [17]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)

<module 'source.util' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/source/util.py'>

In [48]:
T, P1, P2 = 10, 5, 5
g = game.zs_game(util.gen_tar_with_len(T), 10) #targets_dict[10][0]
d = holmes.HOLMES(g, 0, 1, L=1, exploration=False)
#d = b2bw2w.BeliefMaximizer(g, 0, 1)
targets = util.gen_tar_with_len(T)
profiles = util.gen_profiles(targets, [(attackers.UnknownStochasticAttacker, 1),
                                       (attackers.StackelbergAttacker, 1),
                                       (attackers.SUQR, P1),
                                       (attackers.StochasticAttacker, P2)])
#a = np.random.choice(profiles, 1, replace=False)[0]
#a = attackers.StackelbergAttacker(g, 1)
a = profiles[-1]
a.game = g
g.set_players([d], [a], profiles)

In [18]:
T, P1, P2 = 3, 5, 5
g = game.zs_game(util.gen_tar_with_len(T), 10) #targets_dict[10][0]
d = holmes.HOLMES(g, 0, 1, L=1, exploration=False)
#d = b2bw2w.BeliefMaximizer(g, 0, 1)
targets = util.gen_tar_with_len(T)
profiles = util.gen_profiles(targets, [(attackers.UnknownStochasticAttacker, 1),
                                       (attackers.StackelbergAttacker, 1),
                                       (attackers.SUQR, P1),
                                       (attackers.StochasticAttacker, P2)])
#a = np.random.choice(profiles, 1, replace=False)[0]
#a = attackers.StackelbergAttacker(g, 1)
a = profiles[-1]
a.game = g
g.set_players([d], [a], profiles)

In [19]:
e = runner.Experiment(deepcopy(g), env = "rt")

In [24]:
e.run_interaction()

insert a valid strategy
4
insert a valid strategy
23
insert a valid strategy
pupu?
insert a valid strategy
3
insert a valid strategy
2


In [25]:
e.game.strategy_history

[{0: (0.23055815818396624, 0.40286718232333191, 0.36657465949270185),
  1: [0, 1, 0]},
 {0: (0, 1, 0), 1: [1, 0, 0]},
 {0: (0, 1, 0), 1: [0, 0, 1]}]

In [97]:
i = 0

In [166]:
i +=1

In [38]:
e = runner.Experiment(deepcopy(g))

In [41]:
#start = time.time()
e.run_interaction()
print(util.game_str(e.game))
print(e.agent.sel_arm, e.game.strategy_history[-1][0])
#print(time.time() - start)

 	 	□	 	 	
0.527	0.699	0.698	0.746	0.624	
 	 	 	 	△	
----------------------------------------
 	 	 	 	□	
0.527	0.699	0.698	0.746	0.624	
 	 	 	 	△	
----------------------------------------
 	 	 	 	□	
0.527	0.699	0.698	0.746	0.624	
 	 	 	 	△	
usto_defV21 [0, 0, 0, 0, 1]
